In [1]:
import pandas as pd

dataset = pd.read_csv('arquivos/supermarket_sales.csv')
dataset.head()

,Invoice ID,Branch,City,Customer type,Gender,Product line,Unit price,Quantity,Tax 5%,Total,Date,Time,Payment,cogs,gross margin percentage,gross income,Rating
0,750-67-8428,A,Yangon,Member,Female,Health and beauty,74.69,7,26.1415,548.9715,1/5/2019,13:08,Ewallet,522.83,4.761905,26.1415,9.1
1,226-31-3081,C,Naypyitaw,Normal,Female,Electronic accessories,15.28,5,3.8200,80.2200,3/8/2019,10:29,Cash,76.40,4.761905,3.8200,9.6
2,631-41-3108,A,Yangon,Normal,Male,Home and lifestyle,46.33,7,16.2155,340.5255,3/3/2019,13:23,Credit card,324.31,4.761905,16.2155,7.4
3,123-19-1176,A,Yangon,Member,Male,Health and beauty,58.22,8,23.2880,489.0480,1/27/2019,20:33,Ewallet,465.76,4.761905,23.2880,8.4
4,373-73-7910,A,Yangon,Normal,Male,Sports and travel,86.31,7,30.2085,634.3785,2/8/2019,10:37,Ewallet,604.17,4.761905,30.2085,5.3


In [2]:
dataset['Rating'].mean()

np.float64(6.9727)

In [3]:
dataset['Rating'].mean()

np.float64(6.9727)

In [4]:
import openai
from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv())

client = openai.Client()

## Cria o assistant

In [5]:
file = client.files.create(
    file=open('arquivos/supermarket_sales.csv', 'rb'),
    purpose='assistants'
)

In [6]:
file.id

'file-zyHspS1SP3J9GT8FLYBrIayV'

In [7]:
# import pandas as pd

# dataset = pd.read_csv('arquivos/supermarket_sales.csv')
# dataset.head()


In [8]:
assistant = client.beta.assistants.create(
    name="Analista Fianceiro",
    instructions="Você é um analista financeiro de um supermercado. Você utiliza os dados .csv relativo às vendas \
        do supermercado para realizar as suas análises",
    tools=[{'type': 'code_interpreter'}],
    tool_resources={'code_interpreter': {'file_ids': [file.id]}},
    model='gpt-4o'
)


# ```python
# assistant = client.beta.assistants.create(
# ```
# - **Propósito:** Esta linha inicia a criação de um novo assistente usando o método `client.beta.assistants.create`.
# - **Explicação:** O objeto `assistant` armazenará a instância do assistente que você está criando. Este é tipicamente o primeiro passo para configurar um assistente personalizado usando uma API.

# ```python
#     name="Analista Fianceiro",
# ```
# - **Propósito:** Especifica o nome do assistente.
# - **Explicação:** Aqui, você está nomeando o assistente como "Analista Financeiro". Este nome é útil para identificar o assistente dentro do seu projeto ou aplicação.

# ```python
#     instructions="Você é um analista financeiro de um supermercado. Você utiliza os dados .csv relativo às vendas \
#         do supermercado para realizar as suas análises",
# ```
# - **Propósito:** Fornece instruções para o assistente.
# - **Explicação:** Esta string dá contexto e orientação ao assistente sobre como ele deve se comportar. Você está instruindo o assistente a agir como um analista financeiro de um supermercado e a utilizar dados em formato CSV relacionados às vendas para realizar suas análises. A barra invertida (`\`) no final da linha permite que a string continue na próxima linha sem quebrar o código.

# ```python
#     tools=[{'type': 'code_interpreter'}],
# ```
# - **Propósito:** Especifica as ferramentas que o assistente deve ter acesso.
# - **Explicação:** Aqui, você está dando ao assistente acesso a uma ferramenta chamada `code_interpreter`. Esta é uma ferramenta especializada que permite ao assistente executar código, tipicamente para realizar análises de dados, cálculos ou outras tarefas programáveis.

# ```python
#     tool_resources={'code_interpreter': {'file_ids': [file.id]}},
# ```
# - **Propósito:** Vincula recursos (como arquivos) às ferramentas especificadas.
# - **Explicação:** Esta linha está associando um arquivo à ferramenta `code_interpreter`. O parâmetro `file_ids` contém uma lista de IDs de arquivos que o interpretador de código pode usar. O `file.id` provavelmente é o ID do arquivo CSV que você carregou ou criou anteriormente. Isso permite que o interpretador de código acesse os dados nesse arquivo durante as operações do assistente.

# ```python
#     model='gpt-4o'
# ```
# - **Propósito:** Especifica o modelo que o assistente usará.
# - **Explicação:** O assistente está configurado para usar o modelo `gpt-4o`, que é uma variante específica do modelo GPT-4. Esta versão do modelo provavelmente é otimizada ou adaptada para casos de uso específicos, embora mais informações sejam necessárias para explicar completamente suas características.

# No geral, este bloco de código configura um assistente personalizado para atuar como um analista financeiro usando o modelo GPT-4o, com ferramentas e recursos específicos disponíveis para realizar tarefas como a análise de dados de vendas de um supermercado a partir de um arquivo CSV.

In [9]:
# assistant = client.beta.assistants.create(
#     name="Analista Fianceiro Supermercados Asimov",
#     instructions="Você é um analista financeiro de um supermercado. Você utiliza os dados .csv relativo às vendas \
#         do supermercado para realizar as suas análises",
#     tools=[{'type': 'code_interpreter'}],
#     file_ids=[file.id],
#     model='gpt-4-turbo-preview'
# )

## Cria uma thread

In [10]:
thread = client.beta.threads.create()

## Adiciona mensagem a thread criada

In [14]:
pergunta = 'Qual é o rating médio das vendas do nosso supermercado?'
# pergunta = 'Gere um gráfico pizza com o percentual de vendas por meio de pagamento'

messages = client.beta.threads.messages.create(
    thread_id=thread.id,
    role='user',
    content= pergunta
)

In [15]:
pergunta

'Qual é o rating médio das vendas do nosso supermercado?'

## Roda a thread no assistant

In [16]:
run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant.id,
    instructions='O nome do usuário é Sergio.'
)

## Aguarda a thread rodar

In [17]:
import time

while run.status in ['queued', 'in_progress', 'cancelling']:
    time.sleep(1)
    run = client.beta.threads.runs.retrieve(
        thread_id=thread.id,
        run_id=run.id
    )

print(run.status)

completed


## Verifica a resposta

In [18]:
if run.status == 'completed':
    messages = client.beta.threads.messages.list(
        thread_id=thread.id
    )
    print(messages)
else:
    print('Erro', run.status)

SyncCursorPage[Message](data=[Message(id='msg_fhuYkgYxqsoKyU4ez45GknVi', assistant_id='asst_cxtezbL8sqWZVUKq1uDkuiki', attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='Conforme os dados carregados e analisados, o rating médio das vendas do seu supermercado é aproximadamente **6,97**.\n\nCaso precise de mais alguma análise ou informações específicas, sinta-se à vontade para pedir!'), type='text')], created_at=1724165731, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='assistant', run_id='run_bp3btDUGfo616uxPZRADvEX6', status=None, thread_id='thread_bievOHlPxvQTHUFS7mym0s6v'), Message(id='msg_EsofrVmhzsohxIM3OPRkhmfl', assistant_id=None, attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='Qual é o rating médio das vendas do nosso supermercado?'), type='text')], created_at=1724165716, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.messag

In [13]:
messages.data[0].content[0]

TextContentBlock(text=Text(annotations=[], value='Para gerar um gráfico do tipo pizza (pie chart) mostrando o percentual de vendas por meio de pagamento, precisaremos de alguns detalhes adicionais:\n\n1. Os diferentes meios de pagamento que você quer incluir no gráfico.\n2. O total de vendas (ou porcentagem) para cada meio de pagamento.\n\nSe você puder me fornecer essas informações, posso criar o gráfico para você. Se não tiver os dados exatos, também posso criar um exemplo com dados fictícios para ilustrar como seria.'), type='text')

## Analisando os passos do modelo

In [14]:
run_steps = client.beta.threads.runs.steps.list(
  thread_id=thread.id,
  run_id=run.id
)

In [15]:
for step in run_steps.data[::-1]:
    print('======= Step >', step.step_details.type)
    if step.step_details.type == 'tool_calls':
        for tool_call in step.step_details.tool_calls:
            print('```')
            print(tool_call.code_interpreter.input)
            print('```')
            if tool_call.code_interpreter.outputs[0].type == 'logs':
                print('Result')
                print(tool_call.code_interpreter.outputs[0].logs)
    if step.step_details.type == 'message_creation':
        message = client.beta.threads.messages.retrieve(
            thread_id=thread.id,
            message_id=step.step_details.message_creation.message_id
        )
        if message.content[0].type == 'text':
            print(message.content[0].text.value)

        if message.content[0].type == 'image_file':
            file_id = message.content[0].image_file.file_id
            image_data = client.files.content(file_id)

            with open(f'arquivos/{file_id}.png', 'wb') as file:
                file.write(image_data.read())

            import matplotlib.pyplot as plt
            import matplotlib.image as mpimg

            img = mpimg.imread(f'arquivos/{file_id}.png')
            fig, ax = plt.subplots()
            ax.set_axis_off()
            ax.imshow(img)
            plt.show()
        

======= Step > message_creation
Para gerar um gráfico do tipo pizza (pie chart) mostrando o percentual de vendas por meio de pagamento, precisaremos de alguns detalhes adicionais:

1. Os diferentes meios de pagamento que você quer incluir no gráfico.
2. O total de vendas (ou porcentagem) para cada meio de pagamento.

Se você puder me fornecer essas informações, posso criar o gráfico para você. Se não tiver os dados exatos, também posso criar um exemplo com dados fictícios para ilustrar como seria.
